In [1]:
#TRABAJO HECHO POR SANTIAGO RIVERA ESTRADA
#ASESORES JULIANA MORENO Y JOHN FREDY OCHOA
#NEUROCO (GRUPO DE NEUROCIENCIAS COMPUTACIONALES)-UNIVERSIDAD DE ANTIOQUIA-MEDELLIN,COLOMBIA
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mediapipe as mp

In [3]:
# Clase Detector_Pose
class Detector_Pose():
    def __init__(self, mode=False, smooth=True, confianza_dete=0.5, confianza_segui=0.5):
        # Inicialización de la clase
        self.mode = mode  # Modo de la detección (estática o dinámica)
        self.smooth = smooth  # Suavizado de las landmarks
        self.confianza_dete = confianza_dete  # Umbral de confianza para la detección
        self.confianza_segui = confianza_segui  # Umbral de confianza para el seguimiento
        self.smooth = bool(self.smooth)  # Convertir a booleano
        # Importar las librerías y configurar el modelo de detección de pose
        self.mpPose = mp.solutions.pose
        self.mpDraw = mp.solutions.drawing_utils
        self.pose = self.mpPose.Pose(
            static_image_mode=self.mode,
            model_complexity=0 if self.mode else 1,
            smooth_landmarks=self.smooth,
            min_detection_confidence=self.confianza_dete,
            min_tracking_confidence=self.confianza_segui
        )

    def Obtener_posicion(self, img):
        # Método para obtener las posiciones de los puntos de referencia
        self.lm_List = []  # Lista para almacenar las landmarks relevantes
        coordenadas_xyz = []  # Lista para almacenar las coordenadas XYZ de las landmarks relevantes
        imgRGB = cv.cvtColor(img, cv.COLOR_BGR2RGB)  # Convertir imagen a formato RGB
        resul = self.pose.process(imgRGB)  # Procesar la imagen con el modelo de detección de pose
        if resul.pose_landmarks:
            # Si se detectaron landmarks
            for iD, lm in enumerate(resul.pose_landmarks.landmark):
                altura, ancho, canales = img.shape
                if iD == 12 or iD == 14 or iD == 16:
                    # Si la landmark es relevante (hombro, codo o muñeca)
                    cx, cy = int(lm.x * ancho), int(lm.y * altura)  # Calcular las coordenadas en píxeles
                    self.lm_List.append([iD, cx, cy])  # Agregar la landmark a la lista
                    coordenadas_xyz.append([iD, cx, cy, lm.z])  # Agregar coordenadas XYZ a la lista
        return self.lm_List, coordenadas_xyz  # Devolver las listas de landmarks y coordenadas XYZ

    def Obtener_Angulo(self, img, dibujar=True):
        # Método para obtener el ángulo entre tres puntos de referencia relevantes
        lm_List, _ = self.Obtener_posicion(img)  # Obtener la lista de landmarks
        angulo = None  # Inicializar el ángulo
        if len(lm_List) >= 3:
            # Si hay al menos tres landmarks detectadas
            # Obtener las coordenadas de los puntos relevantes
            p1 = np.array(lm_List[0][1:3])  # Punto 1 (hombro derecho)
            p2 = np.array(lm_List[1][1:3])  # Punto 2 (codo derecho)
            p3 = np.array(lm_List[2][1:3])  # Punto 3 (muñeca derecha)
            
            # Calcular los vectores entre los puntos
            vector1 = p1 - p2  # Vector del codo al hombro
            vector2 = p3 - p2  # Vector del codo a la muñeca
            
            # Calcular el ángulo entre los vectores usando np.diff y arccos
            cos_angulo = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
            angulo = np.arccos(cos_angulo)  # Ángulo en radianes
            
            # Convertir a grados
            angulo = np.degrees(angulo)
        
        return angulo  # Devolver el ángulo calculado

In [6]:
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Definir el objeto de detección de poses
detector = Detector_Pose()

# Solicitar la ruta de la carpeta principal
ruta = input("Ruta de la carpeta que contiene los videos, ejemplo : C:\\Users\\santi\\semestre 2024-2\\Videos_Gruneco... Recuerde agregar \\ en cada entrada: ")
carpeta_videos = ruta

# Crear la carpeta de resultados al mismo nivel de la carpeta VIDEOS_SAN_PORTABLES
carpeta_resultados = os.path.join(os.path.dirname(carpeta_videos), 'Resultados')
if not os.path.exists(carpeta_resultados):
    os.makedirs(carpeta_resultados)

# Recorrer todos los archivos en la carpeta y subcarpetas de videos
for carpeta_raiz, subcarpetas, archivos in os.walk(carpeta_videos):
    subcarpetas.sort()  # Ordenar las subcarpetas alfabéticamente
    archivos.sort()  # Ordenar los archivos alfabéticamente

    for archivo in archivos:
        if archivo.endswith('.mp4'):  # Procesar solo archivos de video
            video_path = os.path.join(carpeta_raiz, archivo)
            print(f"Procesando video: {archivo}")
            
            # Cargar el video
            cap = cv.VideoCapture(video_path)
            
            if not cap.isOpened():
                print(f"Error: No se pudo abrir el video {archivo}.")
                continue

            # Inicializar listas para almacenar los datos
            angulos, fotogramas, tiempos = [], [], []
            trayectoria_12_x, trayectoria_12_y = [], []
            trayectoria_14_x, trayectoria_14_y = [], []
            trayectoria_16_x, trayectoria_16_y = [], []
            coordenadas_x, coordenadas_y = [], []
            frame_count = 0
            fps = cap.get(cv.CAP_PROP_FPS)  # Obtener el framerate del video

            # Procesar el video fotograma por fotograma
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                # Aplicar la rutina de detección de poses al fotograma actual
                angulo = detector.Obtener_Angulo(frame)
                angulos.append(angulo)
                fotogramas.append(frame_count)
                tiempo_transcurrido = frame_count / fps
                tiempos.append(tiempo_transcurrido)
                frame_count += 1

                # Obtener posiciones de los puntos de referencia
                lm_List, _ = detector.Obtener_posicion(frame)
                for lm in lm_List:
                    if lm[0] == 12:
                        trayectoria_12_x.append(lm[1])
                        trayectoria_12_y.append(lm[2])
                    elif lm[0] == 14:
                        trayectoria_14_x.append(lm[1])
                        trayectoria_14_y.append(lm[2])
                    elif lm[0] == 16:
                        trayectoria_16_x.append(lm[1])
                        trayectoria_16_y.append(lm[2])

                if lm_List:
                    x, y = lm_List[0][1], lm_List[0][2]
                    coordenadas_x.append(x)
                    coordenadas_y.append(y)

            # Calcular la velocidad angular en grados/segundo
            velocidad_angular_grados = [
                (angulos[i] - angulos[i-1]) / (tiempos[i] - tiempos[i-1]) 
                for i in range(1, len(angulos))
            ]
            velocidad_angular_grados.insert(0, np.nan)  # Insertar un NaN para mantener la longitud

            cap.release()

            # Crear una subcarpeta para guardar los resultados del video procesado
            subcarpeta_resultados = os.path.join(carpeta_resultados, os.path.basename(carpeta_raiz))
            if not os.path.exists(subcarpeta_resultados):
                os.makedirs(subcarpeta_resultados)

            # Guardar gráficas
            plt.figure(figsize=(10, 10))
            plt.subplot(611)
            plt.plot(fotogramas, angulos)
            plt.title('Ángulo en función del fotograma')
            plt.xlabel('Fotograma')
            plt.ylabel('Ángulo (grados)')
            plt.grid(True)

            plt.subplot(612)
            plt.plot(tiempos, angulos)
            plt.title('Ángulo en función del tiempo')
            plt.xlabel('Tiempo (segundos)')
            plt.ylabel('Ángulo (grados)')
            plt.grid(True)

            plt.subplot(613)
            plt.plot(trayectoria_12_x, trayectoria_12_y, label='Hombro derecho (12)')
            plt.plot(trayectoria_14_x, trayectoria_14_y, label='Codo derecho (14)')
            plt.plot(trayectoria_16_x, trayectoria_16_y, label='Muñeca derecha (16)')
            plt.title('Trayectorias de puntos de referencia de la pose')
            plt.xlabel('Coordenada X')
            plt.ylabel('Coordenada Y')
            plt.legend()
            plt.grid(True)

            plt.subplot(614)
            plt.plot(fotogramas, coordenadas_x, label='Coordenada X')
            plt.plot(fotogramas, coordenadas_y, label='Coordenada Y')
            plt.title('Trayectoria del movimiento completo en el tiempo')
            plt.xlabel('Número de fotograma')
            plt.ylabel('Coordenada')
            plt.legend()
            plt.grid(True)

            plt.subplot(615)
            plt.plot(tiempos, trayectoria_16_x, label='Coordenada X de la muñeca')
            plt.plot(tiempos, trayectoria_16_y, label='Coordenada Y de la muñeca')
            plt.title('Trayectoria de la muñeca en el tiempo')
            plt.xlabel('Tiempo (segundos)')
            plt.ylabel('Coordenada')
            plt.legend()

            plt.subplot(616)
            plt.plot(tiempos, velocidad_angular_grados)  # Graficar la velocidad angular en grados/seg
            plt.title('Velocidad Angular en función del Tiempo (Grados/seg)')
            plt.xlabel('Tiempo (segundos)')
            plt.ylabel('Velocidad Angular (grados/seg)')
            plt.grid(True)

            plt.tight_layout()
            plt.savefig(os.path.join(subcarpeta_resultados, f'{archivo}_grafica.png'))  # Guardar la gráfica
            plt.close()

            # Guardar datos en un archivo CSV
            datos = {
                'Fotograma': fotogramas,
                'Tiempo (s)': tiempos,
                'Ángulo (grados)': angulos,
                'Velocidad Angular (grados/seg)': velocidad_angular_grados,
                'Coordenada X': coordenadas_x,
                'Coordenada Y': coordenadas_y
            }
            df = pd.DataFrame(datos)
            df.to_csv(os.path.join(subcarpeta_resultados, f'{archivo}_datos.csv'), index=False)
            
            print(f"Resultados guardados para el video: {archivo}")


Procesando video: Sub-SAN01_ses_V0_task_DTAN_video.mp4
Resultados guardados para el video: Sub-SAN01_ses_V0_task_DTAN_video.mp4
Procesando video: Sub-SAN01_ses_V0_task_DTCT_video.mp4
Resultados guardados para el video: Sub-SAN01_ses_V0_task_DTCT_video.mp4
Procesando video: Sub-SAN01_ses_V0_task_DTF_video.mp4
Resultados guardados para el video: Sub-SAN01_ses_V0_task_DTF_video.mp4
Procesando video: Sub-SAN01_ses_V0_task_DTS1_video.mp4
Resultados guardados para el video: Sub-SAN01_ses_V0_task_DTS1_video.mp4
Procesando video: Sub-SAN01_ses_V0_task_DTS7_video.mp4
Resultados guardados para el video: Sub-SAN01_ses_V0_task_DTS7_video.mp4
Procesando video: Sub-SAN01_ses_V0_task_DTV_video.mp4
Resultados guardados para el video: Sub-SAN01_ses_V0_task_DTV_video.mp4
Procesando video: Sub-SAN01_ses_V0_task_ST1_video.mp4
Resultados guardados para el video: Sub-SAN01_ses_V0_task_ST1_video.mp4
Procesando video: Sub-SAN01_ses_V0_task_ST2_video.mp4
Resultados guardados para el video: Sub-SAN01_ses_V0_tas